In [1]:
%run Omar-Thesis-Common-10.ipynb

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import urllib

#Extracting tweets and and their deep features
start_time = time.time()

target_names = ["comment","deny", "query","support"]
le.fit(target_names)

TrainData,DevData,TestData,GoldData = [] , [], [], []

#Deny_Words = ['not true', 'shut', 'not', "don't agree","impossible",'false','lies']
Deny_Words = ['not true', 'shut', 'not', "don't agree","impossible",'false']
Deny_Words_Set = set(Deny_Words)
Support_Words = ['true', 'exactly','yes','indeed','agree','omg','know']
Support_Words_Set = set(Support_Words)
URL_Words = ['http://','https://']
URL_Words_Set = set(URL_Words)

ID_COMMENT = 0
ID_DENY = 1
ID_QUERY = 2
ID_SUPPORT = 3

DENY_WORD_FEATURE = "deny_word"
SUPPORT_WORD_FEATURE = "support_word"
QUEST_WORD_FEATURE = "quest_word"
HASHTAG_EXISTS_FEATURE = "hashtag_exists"
URL_EXISTS_FEATURE = "url_exists"
SENTIMENT_WORD_FEATURE = "sentiment_word"
POS_WORDS_COUNT = "pos_words_count"
NEG_WORDS_COUNT = "neg_words_count"
NEUT_WORDS_COUNT = "neut_words_count"
IS_REPLY_FEATURE = "is_reply_tweet"
FT_USER_VERIFIED = "user_verified"
FT_SRC_USER_VERIFIED = "src_user_verified"
FT_DIRECT_SRC_USER_VERIFIED = "direct_src_user_verified"
FT_USER_FOLLOWERS_COUNT = "user_followers"
FT_USER_TWEETS_COUNT = "user_tweets"
FT_COSINE_SIM_SRC_TWEET = "cos_sim_src_tweet"
FT_COSINE_SIM_SRC_DIRECT_TWEET = "cos_sim_src_direct_tweet"
FT_RETWEET_RATIO = "ft_retweet_ratio"
FT_USER_FRIENDS_COUNT = "user_friends"
FT_PHOTO_EXISTS = "photo_exists"
FT_DAYS_SINCE_USER_CREATION = "days_since_user_creation"
FT_POS_WORDS_COUNT = "pos_words_count"
FT_NEG_WORDS_COUNT = "neg_words_count"
FT_POS_SENT = "ft_sent"


ATTR_ID = "id"
ATTR_IS_SRC = "is_src"
ATTR_CATEGORY = "category"
ATTR_TEXT = "text"
ATTR_REPLY_STATUS_ID = "in_reply_to_status_id"
ATTR_USER = "user"
ATTR_USER_VERIFIED = "verified"
ATTR_NEW_TEXT = "new_text"
ATTR_FOLLOWERS_COUNT = "followers_count"
ATTR_TWEETS_COUNT = "statuses_count"
ATTR_RETWEET_COUNT = "retweet_count"
ATTR_FRIENDS_COUNT = "friends_count"
ATTR_EXT_ENTITIES = "extended_entities"
ATTR_MEDIA = "media"
ATTR_MEDIA_TYPE = "type"
ATTR_MEDIA_TYPE_PHOTO = "photo"
ATTR_USER_CREATED_AT = "created_at"

currentTime = time.strftime(MY_DATE_FORMAT, gmtime())
currentTime = datetime.strptime(currentTime, MY_DATE_FORMAT)

#Map between each tweet and a list of its replies (to be used by task B)
from collections import defaultdict
TweetRepliesMap = defaultdict(list)

def ExtractTweetFeatures(tweet,src_tweet,direct_src_tweet):
    posWordsCount = 0
    negWordsCount = 0

    tweet[ATTR_NEW_TEXT] =  tweet[ATTR_TEXT]

    #Remove URLs:
    #URLs show something in common between tweets, so removing them produced a worse result.
    #tweet[ATTR_NEW_TEXT] = re.sub(r"http\S+", "", tweet[ATTR_TEXT])

    tweetText = tweet[ATTR_NEW_TEXT].lower()
    #tweetText = tweet[ATTR_NEW_TEXT]

    tokens = tknzr.tokenize(tweetText)
    tokensSet = set(tokens)

    #Extract features
    if questionDetector.IsQuestion( tweet[ATTR_NEW_TEXT]) == True:
        tweet[QUEST_WORD_FEATURE] = 1
    else:
        tweet[QUEST_WORD_FEATURE] = 0

    #if any(word in tweetText for word in Deny_Words):
    if (Deny_Words_Set.intersection(tokensSet)):
        tweet[DENY_WORD_FEATURE] = 1
        #tweet[DENY_WORD_FEATURE] = len(Deny_Words_Set.intersection(tokensSet))
        tweet[SUPPORT_WORD_FEATURE] = 0
    elif (Support_Words_Set.intersection(tokensSet)):
        tweet[SUPPORT_WORD_FEATURE] = 1
        #tweet[SUPPORT_WORD_FEATURE] = len(Support_Words_Set.intersection(tokensSet))
        tweet[DENY_WORD_FEATURE] = 0
    else:
        tweet[DENY_WORD_FEATURE] = 0
        tweet[SUPPORT_WORD_FEATURE] = 0

    #Hashtag
    if (tweetText.find("#")):
        tweet[HASHTAG_EXISTS_FEATURE] = 1
    else:
        tweet[HASHTAG_EXISTS_FEATURE] = 0

    tweet[URL_EXISTS_FEATURE] = 0
    for word in  tokensSet:
        for urlToken in URL_Words_Set:
            if urlToken in word:
                tweet[URL_EXISTS_FEATURE] = 1
                break

    for word in  tokensSet:            
        if word in PosLexiconList:
            posWordsCount += 1
        elif word in NegLexiconList:
            negWordsCount += 1

    tweet[POS_WORDS_COUNT] = posWordsCount
    tweet[NEG_WORDS_COUNT] = negWordsCount
    tweet[NEUT_WORDS_COUNT] = len(tokensSet)-negWordsCount-posWordsCount

    tweet[FT_POS_WORDS_COUNT] = posWordsCount
    tweet[FT_NEG_WORDS_COUNT] = negWordsCount
    if posWordsCount > negWordsCount:
        tweet[SENTIMENT_WORD_FEATURE] = 1
    elif negWordsCount > posWordsCount:
        tweet[SENTIMENT_WORD_FEATURE] = 0
    else:
        tweet[SENTIMENT_WORD_FEATURE] = 0.5

    if tweet[ATTR_REPLY_STATUS_ID]:
        tweet[IS_REPLY_FEATURE] = 1
    else:
        tweet[IS_REPLY_FEATURE] = 0

    if tweet[ATTR_USER][ATTR_USER_VERIFIED] == True:
        tweet[FT_USER_VERIFIED] = 1
    else:
        tweet[FT_USER_VERIFIED] = 0

    if src_tweet[ATTR_USER][ATTR_USER_VERIFIED] == True:
        tweet[FT_SRC_USER_VERIFIED] = 1
    else:
        tweet[FT_SRC_USER_VERIFIED] = 0

    tweet[FT_USER_FOLLOWERS_COUNT] = tweet[ATTR_USER][ATTR_FOLLOWERS_COUNT]
    tweet[FT_USER_TWEETS_COUNT] = tweet[ATTR_USER][ATTR_TWEETS_COUNT]
    tweet[FT_USER_FRIENDS_COUNT] = src_tweet[ATTR_USER][ATTR_FRIENDS_COUNT]


    #Some of the direct src tweets are not found !
    if direct_src_tweet is not None and direct_src_tweet[ATTR_USER][ATTR_USER_VERIFIED] == True:
        tweet[FT_DIRECT_SRC_USER_VERIFIED] = 1
    else:
        tweet[FT_DIRECT_SRC_USER_VERIFIED] = 0

    #Compute the cosine similarity with the src tweet
    SrcTweetTokens = tknzr.tokenize(src_tweet[ATTR_TEXT])
    cosSim = get_cosine(Counter(tokens),Counter(SrcTweetTokens))
    if cosSim > 0.75:
        tweet[FT_COSINE_SIM_SRC_TWEET] = 1
        #print('%r -- %r -> %s' % (tokens,SrcTweetTokens,cosSim))
    else:
        tweet[FT_COSINE_SIM_SRC_TWEET] = 0

    tweet[FT_COSINE_SIM_SRC_DIRECT_TWEET] = 0
    if direct_src_tweet is not None:
        SrcDirectTweetTokens = tknzr.tokenize(direct_src_tweet[ATTR_TEXT])
        cosSim = get_cosine(Counter(tokens),Counter(SrcDirectTweetTokens))
        if cosSim > 0.75:
            tweet[FT_COSINE_SIM_SRC_DIRECT_TWEET] = 1

    if src_tweet[ATTR_RETWEET_COUNT] != 0:
        tweet[FT_RETWEET_RATIO] = tweet[ATTR_RETWEET_COUNT]/src_tweet[ATTR_RETWEET_COUNT]
    else:
        tweet[FT_RETWEET_RATIO] = 0
    #print('Retweet Ratio= %s' % (tweet[FT_RETWEET_RATIO]))

    #Check if photo exists
    tweet[FT_PHOTO_EXISTS] = 0
    if ATTR_EXT_ENTITIES in tweet:
        ExtEntities = tweet[ATTR_EXT_ENTITIES]
        if ExtEntities is not None:
            Media = ExtEntities[ATTR_MEDIA]
            if Media is not None:
                for m in Media:
                    if m[ATTR_MEDIA_TYPE] == ATTR_MEDIA_TYPE_PHOTO:
                        tweet[FT_PHOTO_EXISTS] = 1
                        break

    #Compute number of days since user creation feature
    tweetTime = time.strftime(MY_DATE_FORMAT, time.strptime(tweet[ATTR_USER][ATTR_USER_CREATED_AT],TWITTER_DATE_FORMAT))
    tweetTime = datetime.strptime(tweetTime, MY_DATE_FORMAT)
    delta = currentTime - tweetTime
    tweet[FT_DAYS_SINCE_USER_CREATION] = delta.days  
    
    #Compute Sentiment
    tweet[FT_POS_SENT] = SentAnalyzer.classify(tweet[ATTR_NEW_TEXT])
    
    #to be used by task B: populate the Map between each tweet and a list of its replies
    TweetRepliesMap[src_tweet[ATTR_ID]].append(tweet)
    
    return tweet

def GetTweetsInfo (TaskData,IsTest):
    ResultTweetsList = []
    counter = 0
    for key, value in TaskData.items():       
        tweet = GetTweetObj(key,IsTest)
        if tweet is None:
            continue
            
        print(key)
        print(value)
        print(type(value))
        tweet[ATTR_CATEGORY] = le.transform([value]) #VERSION CHANGE(1): Use List of value instead of value only
        
        if IsTest:
            src_tweet_id = SrcTestTweetsDict[str(tweet[ATTR_ID])]
        else:
            src_tweet_id = SrcTweetsDict[str(tweet[ATTR_ID])]
        src_tweet = GetTweetObj(src_tweet_id,IsTest)
        direct_src_tweet_id = tweet[ATTR_REPLY_STATUS_ID]
        direct_src_tweet = GetTweetObj(str(direct_src_tweet_id),IsTest)
    
        tweet = ExtractTweetFeatures(tweet,src_tweet,direct_src_tweet)
        
        ResultTweetsList.append(tweet)
        
        counter+= 1
        
        #if counter % 100 == 0:
        #    print("%s passed" % (counter))

    return ResultTweetsList

def ExtractTestData (TweetsMap,SrcTwtsMap):
    ResultTweetsList = []
    counter = 0
    
    #print("*TweetsMap: " + str(len(TweetsMap)))
    
    for key, value in TweetsMap.items():    
        tweet = value
        
        src_tweet_id = SrcTestTweetsDict[str(tweet[ATTR_ID])]
        src_tweet = SrcTwtsMap[src_tweet_id]
        direct_src_tweet_id = tweet[ATTR_REPLY_STATUS_ID]
        
        if str(direct_src_tweet_id) in SrcTwtsMap:
            direct_src_tweet = SrcTwtsMap[str(direct_src_tweet_id)]
        elif str(direct_src_tweet_id) in TweetsMap:
            direct_src_tweet = TweetsMap[str(direct_src_tweet_id)]
        else:
            direct_src_tweet = None
        
        tweet = ExtractTweetFeatures(tweet,src_tweet,direct_src_tweet)
        
        ResultTweetsList.append(tweet)

        counter+= 1
        
        #if counter % 100 == 0:
        #    print("%s passed" % (counter))
    #print("*ResultTweetsList: " + str(len(ResultTweetsList)))
    return ResultTweetsList

# def ExtractSentiment(TweetsList,fileName):
#     TweetsSent = []
#     counter = 0
#     for tweet in TweetsList:
        
#         ErrCnt = 0
#         while True:
#             try:
#                 tweetText = tweet[ATTR_NEW_TEXT]
#                 print('hiray1')
#                 data = urllib.parse.urlencode({"text": "%r" % (tweetText) }) 
#                 #print(tweetText)
#                 u = urllib.request.urlopen("http://text-processing.com/api/sentiment/", data.encode('utf8'))
#                 the_page = u.read()
#                 print('hiray2')
#                 strJson = str(the_page)[2:-1]
#                 #print(strJson)
#                 print('hiray3')
#                 sentJSON = json.loads(strJson)
#                 sentJSON["id"] = tweet[ATTR_ID]
#                 print('hiray4')
#                 TweetsSent.append(sentJSON)
#                 print('hiray5')
#                 break
#             except Exception as e:
#                 ErrCnt += 1
#                 print ("error({0}): {1} - {2}".format(e.errno, e.strerror,ErrCnt))
            
#         counter+= 1
#         if counter % 100 == 0:
#             print("%s passed" % (counter))

#     with open(fileName, 'w') as outfile:
#         json.dump(TweetsSent, outfile)
        
TrainData = GetTweetsInfo(TaskATrainData,False)
DevData = GetTweetsInfo(TaskADevData,False)
TestData = ExtractTestData(ReplyTestTweetsMap,SrcTestTweetsMap)
GoldData = GetTweetsInfo(TaskAGoldData,True)
SourceTweetsList = ExtractTestData(SrcTestTweetsMap,SrcTestTweetsMap)
for tw in SourceTweetsList :
    TestData.append(tw)
print("Extracting features took --- %s seconds ---" % (time.time() - start_time))

501760642928635904
comment
<class 'str'>
500270212198174720
comment
<class 'str'>
524971210275565568
comment
<class 'str'>
552836882770690049
comment
<class 'str'>
500289931097296897
support
<class 'str'>
544293230649810944
comment
<class 'str'>
544283032732332032
comment
<class 'str'>
500406679176241153
comment
<class 'str'>
500300182844878848
comment
<class 'str'>
552811591683821568
comment
<class 'str'>
524938343180931073
comment
<class 'str'>
521360786409943041
deny
<class 'str'>
524946532416884736
comment
<class 'str'>
544347456238931968
comment
<class 'str'>
525020658972258305
query
<class 'str'>
552817834628501505
comment
<class 'str'>
524959078229880832
comment
<class 'str'>
553588165408473088
comment
<class 'str'>
544293748377518080
comment
<class 'str'>
553196346375954433
query
<class 'str'>
524957146656821249
comment
<class 'str'>
524992668301262848
deny
<class 'str'>
500294827351621632
comment
<class 'str'>
544307827070615553
support
<class 'str'>
524955418955902976
comment

<class 'str'>
544391456140836864
comment
<class 'str'>
544292691928838144
comment
<class 'str'>
553538493738405888
comment
<class 'str'>
553199418032521216
support
<class 'str'>
500317982799982592
comment
<class 'str'>
544292129972170752
support
<class 'str'>
500320630156189697
support
<class 'str'>
544351912599240704
comment
<class 'str'>
544285415814225921
comment
<class 'str'>
544512676643500033
support
<class 'str'>
544518568633991170
support
<class 'str'>
500298745301729282
query
<class 'str'>
500380991035559937
comment
<class 'str'>
544315647182651392
comment
<class 'str'>
544315413862301697
deny
<class 'str'>
544286041155571712
support
<class 'str'>
499370097698291713
comment
<class 'str'>
544290947404943361
comment
<class 'str'>
544466740332462080
deny
<class 'str'>
553521113733824513
comment
<class 'str'>
553195879692517376
comment
<class 'str'>
553587585126117376
comment
<class 'str'>
524974129930592256
comment
<class 'str'>
552833887261691905
comment
<class 'str'>
5442951125

552833319747194882
comment
<class 'str'>
544764006368411648
comment
<class 'str'>
544507804829351937
comment
<class 'str'>
544517721862725632
comment
<class 'str'>
521353020484096001
support
<class 'str'>
500299463135858688
comment
<class 'str'>
544514420769296384
comment
<class 'str'>
524973660311154688
comment
<class 'str'>
544491585384505344
deny
<class 'str'>
529722000076718082
comment
<class 'str'>
552816523593916416
support
<class 'str'>
524925154611441664
comment
<class 'str'>
553549691129192448
comment
<class 'str'>
500393155066626048
query
<class 'str'>
553594257593106432
comment
<class 'str'>
525063388934967298
comment
<class 'str'>
524960409602650113
comment
<class 'str'>
500355191645171712
comment
<class 'str'>
500306749090324480
comment
<class 'str'>
552806438440292352
comment
<class 'str'>
500281916658958337
comment
<class 'str'>
524993079603130368
comment
<class 'str'>
553504747508879360
deny
<class 'str'>
552812407064899585
comment
<class 'str'>
500394049501544449
comme

500390243716304896
comment
<class 'str'>
544323425993371649
comment
<class 'str'>
500281475141357568
deny
<class 'str'>
544511082472415232
comment
<class 'str'>
524972276345622528
comment
<class 'str'>
553568091980824576
comment
<class 'str'>
544514139784478720
comment
<class 'str'>
524926472432410625
support
<class 'str'>
499690552821284864
query
<class 'str'>
500284524971188224
comment
<class 'str'>
500363740311982081
support
<class 'str'>
544275772416880640
comment
<class 'str'>
553507344932294657
comment
<class 'str'>
524930594464362496
support
<class 'str'>
500280140560338944
support
<class 'str'>
544520507580628992
comment
<class 'str'>
552812099294863362
comment
<class 'str'>
500323593805660160
support
<class 'str'>
553577770983645186
support
<class 'str'>
544352859773755392
deny
<class 'str'>
524965775036387329
support
<class 'str'>
525061628078067712
comment
<class 'str'>
499919010625253376
comment
<class 'str'>
500297892511633408
comment
<class 'str'>
544470190768279553
deny


<class 'str'>
500317273002106880
comment
<class 'str'>
521373767130710016
support
<class 'str'>
524935320786456578
comment
<class 'str'>
544316435435376640
support
<class 'str'>
544297696308518912
support
<class 'str'>
524925539996667904
comment
<class 'str'>
544336733455020032
comment
<class 'str'>
529723119913951232
query
<class 'str'>
524929931634286592
comment
<class 'str'>
499921932645433344
comment
<class 'str'>
524969935383326721
comment
<class 'str'>
544318876255653888
deny
<class 'str'>
544381350694752256
comment
<class 'str'>
553555326125895680
comment
<class 'str'>
553555095317929985
comment
<class 'str'>
553585472145211392
comment
<class 'str'>
524953683172524032
deny
<class 'str'>
553615282435289088
comment
<class 'str'>
500289655829712896
support
<class 'str'>
524945130113863680
comment
<class 'str'>
544295739644342272
comment
<class 'str'>
552809852834418688
comment
<class 'str'>
525057081515061248
support
<class 'str'>
500329247148609536
deny
<class 'str'>
5249415381119

...

<class 'str'>
553204674766331904
comment
<class 'str'>
544403194991099904
support
<class 'str'>
524949525644201985
comment
<class 'str'>
500379521900503040
support
<class 'str'>
553553255255781376
query
<class 'str'>
553591055783051264
support
<class 'str'>
500702640305868801
support
<class 'str'>
524976050481422336
comment
<class 'str'>
525080429410664448
comment
<class 'str'>
524947674164760577
support
<class 'str'>
524957310024945665
comment
<class 'str'>
500448504058093568
comment
<class 'str'>
553189559149035520
comment
<class 'str'>
544522210388746240
comment
<class 'str'>
552808282440859648
comment
<class 'str'>
553188299758911488
comment
<class 'str'>
552833272724852736
comment
<class 'str'>
553559527216656385
query
<class 'str'>
553577189837258752
comment
<class 'str'>
544482475410337793
comment
<class 'str'>
553188436178264066
comment
<class 'str'>
500399140572897280
comment
<class 'str'>
553109367042887680
support
<class 'str'>
524976422377775104
comment
<class 'str'>
524960

544302181848195073
comment
<class 'str'>
544298332005613569
comment
<class 'str'>
521378427681898498
deny
<class 'str'>
553199295588212736
comment
<class 'str'>
524967686284255232
comment
<class 'str'>
552823438457311232
deny
<class 'str'>
500369701240066048
comment
<class 'str'>
544283521842708480
support
<class 'str'>
524995006088896513
comment
<class 'str'>
499449102908653570
comment
<class 'str'>
524949153424879617
comment
<class 'str'>
544288694497071104
comment
<class 'str'>
544269749405097984
support
<class 'str'>
500284241985683456
comment
<class 'str'>
500379221504442370
query
<class 'str'>
544515757069975552
comment
<class 'str'>
544282661909323776
comment
<class 'str'>
544331611081162753
comment
<class 'str'>
544540884952694784
comment
<class 'str'>
500338566229401600
comment
<class 'str'>
552811444119797761
comment
<class 'str'>
500297784134606849
comment
<class 'str'>
552819494372921346
comment
<class 'str'>
577287160329465856
comment
<class 'str'>
544485799149187072
comme

500285535882338304
comment
<class 'str'>
758293999589396480
comment
<class 'str'>
498293963183652864
comment
<class 'str'>
524956095841050624
comment
<class 'str'>
544295537181069312
query
<class 'str'>
764990138262822912
comment
<class 'str'>
775150410000719874
comment
<class 'str'>
544324890547851264
comment
<class 'str'>
524938161072242688
query
<class 'str'>
553573025015422976
comment
<class 'str'>
524996856431251456
comment
<class 'str'>
552802854453641216
comment
<class 'str'>
769044993768034305
query
<class 'str'>
763128569929330689
comment
<class 'str'>
524925268553904128
comment
<class 'str'>
770171945329405952
comment
<class 'str'>
758297808030105601
comment
<class 'str'>
758293788427218944
comment
<class 'str'>
581302968487571456
comment
<class 'str'>
524923813931540480
comment
<class 'str'>
580368657450201088
comment
<class 'str'>
498299548625018880
support
<class 'str'>
544364347795836928
comment
<class 'str'>
758346105415344128
query
<class 'str'>
500301689862098946
comme

comment
<class 'str'>
524951904212615168
comment
<class 'str'>
775086109499981825
comment
<class 'str'>
769991003138887681
comment
<class 'str'>
524965748268343296
comment
<class 'str'>
770000139779964928
comment
<class 'str'>
775075528554389504
support
<class 'str'>
524979403403444224
comment
<class 'str'>
770108595518210048
comment
<class 'str'>
801684107616890880
support
<class 'str'>
524950833088458752
comment
<class 'str'>
544275563204976640
comment
<class 'str'>
524962458864476161
comment
<class 'str'>
770001251920711682
support
<class 'str'>
553562249689067520
support
<class 'str'>
581308438036135936
comment
<class 'str'>
581915507391139840
comment
<class 'str'>
544301711003615232
query
<class 'str'>
758329686531399681
comment
<class 'str'>
775046242858627072
support
<class 'str'>
581297818528141313
query
<class 'str'>
553495937432432640
comment
<class 'str'>
767738307828645888
comment
<class 'str'>
758591233446907904
comment
<class 'str'>
581162461807685632
comment
<class 'str'

comment
<class 'str'>
770003175868858369
comment
<class 'str'>
581296168711356416
support
<class 'str'>
758445313937985536
comment
<class 'str'>
771137662304522240
comment
<class 'str'>
553561890362630144
comment
<class 'str'>
553562312381333504
comment
<class 'str'>
769990325909786624
comment
<class 'str'>
774995470737432576
comment
<class 'str'>
775024687612997632
comment
<class 'str'>
759003577977217024
comment
<class 'str'>
581302905812176896
comment
<class 'str'>
758460884217847809
deny
<class 'str'>
524965639677825024
comment
<class 'str'>
767727188800000000
comment
<class 'str'>
581157526391119872
comment
<class 'str'>
544344211038814208
comment
<class 'str'>
769990483045126144
comment
<class 'str'>
774994748092223493
comment
<class 'str'>
767749278399881216
comment
<class 'str'>
758366992319512576
comment
<class 'str'>
774992285557678080
support
<class 'str'>
581157167220277248
support
<class 'str'>
581309953085865984
comment
<class 'str'>
758324563587567617
comment
<class 'str

In [3]:
#Extracting data into lists for ease of access
TrainTweetsText = []
TrainTweetsCategories = []
TrainTweetsIDs = []
for item in TrainData:
    TrainTweetsIDs.append(item[ATTR_ID])
    TrainTweetsText.append(item[ATTR_NEW_TEXT])
    TrainTweetsCategories.append(item[ATTR_CATEGORY][0]) #VERSION CHANGE (result of 1): Use List of value instead of value only
    
DevTweetsText = []
DevTweetsCategories = []
DevTweetsIDs = []
for item in DevData:
    DevTweetsIDs.append(item[ATTR_ID])
    DevTweetsText.append(item[ATTR_NEW_TEXT])
    DevTweetsCategories.append(item[ATTR_CATEGORY])

TestTweetsText = []
TestTweetsIDs = []
for item in TestData:
    TestTweetsIDs.append(item[ATTR_ID])
    TestTweetsText.append(item[ATTR_NEW_TEXT])
    
GoldTweetsText = []
GoldTweetsCategories = []
GoldTweetsIDs = []
for item in GoldData:
    GoldTweetsIDs.append(item[ATTR_ID])
    GoldTweetsText.append(item[ATTR_NEW_TEXT])
    GoldTweetsCategories.append(item[ATTR_CATEGORY])

In [167]:
#Transform the text into word vector
from nltk import word_tokenize

USE_PoS = False
USE_TFIDF = False

if not USE_PoS: 
    if USE_TFIDF:
        print("Using TfidfVectorizer")
        tfidf_vect = TfidfVectorizer(ngram_range=(1,1),stop_words =StopWords,lowercase = False)
        X_train_counts = tfidf_vect.fit_transform(TrainTweetsText)
        X_dev_counts = tfidf_vect.transform(DevTweetsText)
        X_test_counts = tfidf_vect.transform(TestTweetsText)
        X_gold_counts = tfidf_vect.transform(GoldTweetsText)
    else:
        print("Using CountVectorizer")
        #Using Count vectorizer with out PoS
        #count_vect = CountVectorizer(ngram_range=(1,1))
        #count_vect = CountVectorizer(ngram_range=(1,1),stop_words ="english")
        #Stemming causes worse performance
        #count_vect = CountVectorizer(ngram_range=(1,1),stop_words =StopWords,tokenizer=tokenize_stem,lowercase = False)
        count_vect = CountVectorizer(ngram_range=(1,1),stop_words =StopWords,lowercase = False)
        X_train_counts = count_vect.fit_transform(TrainTweetsText)
        #Extract the basic words features for dev data
        X_dev_counts = count_vect.transform(DevTweetsText)
        
        X_test_counts = count_vect.transform(TestTweetsText)
        X_gold_counts = count_vect.transform(GoldTweetsText)
else:
    #Using Dict Vectorizer with PoS
    TrainTweetsTextPoS = []
    for tweetText in TrainTweetsText:
        tokensList = [i for i in word_tokenize(tweetText.lower()) if i not in StopWords]
        PoSTokens = nltk.pos_tag(tokensList) #PoSTokens is a tuple
        TrainTweetsTextPoS.append(dict((y, x) for x, y in PoSTokens))
    
    X_train_counts = dict_vect.fit_transform(TrainTweetsTextPoS)
    
    DevTweetsTextPoS = []
    for tweetText in DevTweetsText:
        tokensList = [i for i in word_tokenize(tweetText.lower()) if i not in StopWords]
        PoSTokens = nltk.pos_tag(tokensList)
        DevTweetsTextPoS.append(dict((y, x) for x, y in PoSTokens))
        
    X_dev_counts = dict_vect.transform(DevTweetsTextPoS)
    
    TestTweetsTextPoS = []
    for tweetText in TestTweetsText:
        tokensList = [i for i in word_tokenize(tweetText.lower()) if i not in StopWords]
        PoSTokens = nltk.pos_tag(tokensList)
        TestTweetsTextPoS.append(dict((y, x) for x, y in PoSTokens))
        
    X_test_counts = dict_vect.transform(TestTweetsTextPoS)
    
    GoldTweetsTextPoS = []
    for tweetText in GoldTweetsText:
        tokensList = [i for i in word_tokenize(tweetText.lower()) if i not in StopWords]
        PoSTokens = nltk.pos_tag(tokensList)
        GoldTweetsTextPoS.append(dict((y, x) for x, y in PoSTokens))
        
    X_gold_counts = dict_vect.transform(GoldTweetsTextPoS)

trainFts = sp.csr_matrix(np.zeros((len(TrainData),0)))
devFts = sp.csr_matrix(np.zeros((len(DevData),0)))
testFts = sp.csr_matrix(np.zeros((len(TestData),0)))
goldFts = sp.csr_matrix(np.zeros((len(GoldData),0)))

addWordVecFeatures = False

if addWordVecFeatures:
    trainFts = X_train_counts
    devFts = X_dev_counts
    testFts = X_test_counts
    goldFts = X_gold_counts


#Print stats
# print (len(TrainTweetsText))
# print (type(trainFts))
# print (trainFts.shape)
# print (devFts.shape)
# print (testFts.shape)
# print (goldFts.shape)

#print(type(trainFts))
#print(trainFts.toarray())

# count = 0
# for f in trainFts.toarray():
#     if f == 1:
#         count = count + 1

# allCount = 0
# for item in TrainData:
#     if item["category"] == ID_QUERY:
#         allCount = allCount + 1
# print("All Queries: " + str(allCount))
# print("Considered questions: " + str(count))

Using CountVectorizer


In [168]:
#Fooling around: Print word features

#if not USE_PoS: 
    #print(count_vect.get_feature_names())
#else:
    #print(dict_vect.get_feature_names())



In [169]:
#Feature selection
CHI_FS_ON = 0
VT_FS_ON = 0
DR_SVD_ON = 0

plt.close()

if CHI_FS_ON:
    print ('Chi-Squared Feature Selection')
    FeaturesCount = 9687
    #FeaturesCount = 9650

    ch2 = SelectKBest(chi2, k=FeaturesCount)

    print (trainFts.shape)

    trainFts = ch2.fit_transform(trainFts, TrainTweetsCategories)
    devFts = ch2.transform(devFts)
    goldFts = ch2.transform(goldFts)

    print (trainFts.shape)


from sklearn.feature_selection import VarianceThreshold

if VT_FS_ON:
    print ('Variance Threshold Feature Selection')
    
    #vtFT = VarianceThreshold(threshold=(0.00023725))
    #vtFT = VarianceThreshold(threshold=(0.2))
    vtFT = VarianceThreshold(0.00025)
    
    print (trainFts.shape)
    
    trainFts = vtFT.fit_transform(trainFts)
    
    print(vtFT.variances_)
    print(min(vtFT.variances_))
    print(max(vtFT.variances_))
    
    _,ax = plt.subplots()
    bins = np.linspace(0.00030,0.008, 1000)
    ax.hist(vtFT.variances_,bins)
        
    devFts = vtFT.transform(devFts)
    goldFts = vtFT.transform(goldFts)
    
    print (trainFts.shape)
    
if DR_SVD_ON:
    svd = TruncatedSVD(n_components=200, n_iter=7, random_state=42)
    svd.fit(trainFts)
    #print(svd.explained_variance_ratio_) 
    
    print (trainFts.shape)
    #print(type(trainFts))
    trainFts = sp.csr_matrix(svd.transform(trainFts))
    devFts = sp.csr_matrix(svd.transform(devFts))
    goldFts = sp.csr_matrix(svd.transform(goldFts))
    
    #print(type(trainFts))
    print (trainFts.shape)

In [170]:
#Add additional features
def AddBinaryFeatureToVector (featureName,features,data,scale=False):
    NewFeatureArr = np.zeros((len(data),1))
    i =0
    for item in data:
        NewFeatureArr[i] = item[featureName]
        i = i + 1
    
    if scale:
        NewFeatureArr = preprocessing.scale(NewFeatureArr)
    
    features = sp.hstack((features, NewFeatureArr), format='csr')
    
    return features

def AddFeature(trainFts,devFts,testFts,goldFts,attrName,scale=False):
    trainFts = AddBinaryFeatureToVector(attrName,trainFts,TrainData,scale)
    devFts = AddBinaryFeatureToVector(attrName,devFts,DevData,scale)
    testFts = AddBinaryFeatureToVector(attrName,testFts,TestData,scale)
    goldFts = AddBinaryFeatureToVector(attrName,goldFts,GoldData,scale)
    return (trainFts,devFts,testFts,goldFts)

trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,QUEST_WORD_FEATURE)
trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,DENY_WORD_FEATURE)
trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,SUPPORT_WORD_FEATURE)
#No use for this
#trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,HASHTAG_EXISTS_FEATURE)

trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,URL_EXISTS_FEATURE)

trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,IS_REPLY_FEATURE)

#Sentiment features
trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,SENTIMENT_WORD_FEATURE)
#trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_POS_WORDS_COUNT,True)
#trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_NEG_WORDS_COUNT,True)
#trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_POS_SENT)

#Accuracy is bad when including User Verified feature, but with the root or direct src users, it gets better !
#trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_USER_VERIFIED)
trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_SRC_USER_VERIFIED)
trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_DIRECT_SRC_USER_VERIFIED)

#User's Followers count
trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_USER_FOLLOWERS_COUNT,True)

#User's tweets count (not useful for act of speech recognition)
#trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_USER_TWEETS_COUNT,True)

#User's friends count (not useful)
#trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_USER_FRIENDS_COUNT,True)

#Cosine similarity of the sentance with the root source tweet
trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_COSINE_SIM_SRC_TWEET)

#Cosine similarity of the sentance with the direct tweet it's replying to
#trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_COSINE_SIM_SRC_DIRECT_TWEET)

#Retweet ratio (not useful for act of speech recognition)
#trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_RETWEET_RATIO)

#Photo existance
#trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_PHOTO_EXISTS)

#Days since user creation
#trainFts,devFts,testFts,goldFts = AddFeature(trainFts,devFts,testFts,goldFts,FT_DAYS_SINCE_USER_CREATION)

print (len(TrainTweetsText))
print (trainFts.shape)
print (devFts.shape)
print (testFts.shape)
print (goldFts.shape)
#print(StopWords)

4214
(4214, 10)
(281, 10)
(1049, 10)
(1049, 10)


In [171]:
#Choose classifier
print('Choosing classifier')

#Better: 0.34
from sklearn.linear_model import SGDClassifier
sdgClassifier = SGDClassifier()

#Better: 0.33
from sklearn.svm import LinearSVC
linearSVCClassifier = LinearSVC()
#clf = LinearSVC(class_weight="balanced") 

# from sklearn.naive_bayes import MultinomialNB
# clf = MultinomialNB()

from sklearn.ensemble import RandomForestClassifier
randomForestClassifier = RandomForestClassifier()

from sklearn.linear_model import LogisticRegression
logisticRegressionClassifier = LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
decisionTreeClassifier = DecisionTreeClassifier()

from sklearn.neural_network import MLPClassifier
mlpClassifier = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100), random_state=1)

from sklearn.ensemble import GradientBoostingClassifier
gradientBoostClassifier = GradientBoostingClassifier()

from sklearn.tree import DecisionTreeClassifier
cart = DecisionTreeClassifier()
from sklearn.ensemble import BaggingClassifier
baggingClassifier = BaggingClassifier(base_estimator=cart,n_estimators=50)

from sklearn.tree import DecisionTreeClassifier
cart = DecisionTreeClassifier()
from sklearn.ensemble import AdaBoostClassifier
adaBoostClassifier = AdaBoostClassifier(base_estimator=cart,n_estimators=100)

#Voting Classifier
from sklearn.ensemble import RandomForestClassifier
rfClf = RandomForestClassifier()
from sklearn.linear_model import LogisticRegression
lrClf = LogisticRegression()
from sklearn.neural_network import MLPClassifier
mlpClf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100), random_state=1)
from sklearn.svm import LinearSVC
svmClf = LinearSVC()
from sklearn.tree import DecisionTreeClassifier
cart = DecisionTreeClassifier()
from sklearn.ensemble import BaggingClassifier
bagClf = BaggingClassifier(base_estimator=cart,n_estimators=50)
from sklearn.tree import DecisionTreeClassifier
cart = DecisionTreeClassifier()
from sklearn.ensemble import AdaBoostClassifier
adaClf = AdaBoostClassifier(base_estimator=cart,n_estimators=100)
from sklearn.ensemble import GradientBoostingClassifier
grClf = GradientBoostingClassifier()
from sklearn.ensemble import VotingClassifier
votingClassifier = VotingClassifier(estimators=[
         ('rfClf', rfClf),('grClf',grClf), ('lrClf', lrClf), ('mlpClf', mlpClf),('svmClf',svmClf),('bagClf',bagClf),('adaClf',adaClf) ],
         voting='hard')

#Choose current classifier
clf = votingClassifier

print('Chose classifier')

Choosing classifier
Chose classifier


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [172]:
#Cross Validation using train data

'''
Quoted from SemEval:
The evaluation of the SDQC task needs to be more careful, as the distribution of the categories is clearly skewed 
towards comments. Given that comments are the least helpful type of message towards establishing the veracity of a 
rumour and the most populous, the evaluation metric needs to reward systems that perform well when classifying support, 
denial, and querying. This will be achieved through macroaveraged F1 aggregated for support, denial, and querying, 
disregarding the performance on comments. Individual S, D, Q and C scores will be given in the final report.
'''

cross_validate = 0

if cross_validate == 1:
    folds = 10

    resultsArr = np.array(list(TrainTweetsCategories), dtype=int)

    scorer = make_scorer(fbeta_score,greater_is_better=True,labels=[ID_DENY,ID_QUERY,ID_SUPPORT],average="macro",beta=2)

    scores = cross_validation.cross_val_score(clf, trainFts, resultsArr, cv=folds,scoring=scorer)

    print(scores)
    print("Macro Averaged F1 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

    scorer = make_scorer(fbeta_score,greater_is_better=True,labels=[ID_DENY,ID_QUERY,ID_SUPPORT],average="weighted",beta=2)

    scores = cross_validation.cross_val_score(clf, trainFts, resultsArr, cv=folds,scoring=scorer)
    print(scores)
    print("Weighted Averaged F1 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

    scores = cross_validation.cross_val_score(clf, trainFts, resultsArr, cv=folds)
    print(scores)
    print("Normal Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [173]:
import sys
sys.version
sys.version_info

import sklearn
import nltk
print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The nltk version is 3.2.4.
The scikit-learn version is 0.19.0.


In [174]:

def testWithDevData(clfName,classifier):

    #Train Classifier
    import numpy as np
    array = np.array(list(TrainTweetsCategories), dtype=int)

    classifier.fit(trainFts,array)

    #Test Classifier
    predicted = classifier.predict(devFts)
    #for doc, category in zip(DevData.keys(), predicted):
    #    print('%r => %s' % (doc, category))


    #Evaluate Classifier using dev data

    resultsArr = np.array(list(DevTweetsCategories), dtype=int)
    print (clfName + " Accuracy: " + str(np.mean(predicted == resultsArr)))

    print("---------------------------------------------------------------------------Without Comments")

    print(metrics.classification_report(resultsArr, predicted,target_names=target_names[1:],labels=[ID_DENY,ID_QUERY,ID_SUPPORT]))

    print("*Macro F1 Average*: " + str(metrics.f1_score(resultsArr, predicted,average='macro',labels=[ID_DENY,ID_QUERY,ID_SUPPORT])))
    print("Micro F1 Average: " + str(metrics.f1_score(resultsArr, predicted,average='micro',labels=[ID_DENY,ID_QUERY,ID_SUPPORT])))
    print("Weighted F1 Average: " + str(metrics.f1_score(resultsArr, predicted,average='weighted',labels=[ID_DENY,ID_QUERY,ID_SUPPORT])))

    print("---------------------------------------------------------------------------Includng Comments")

    print(metrics.classification_report(resultsArr, predicted,target_names=target_names,labels=[ID_COMMENT,ID_DENY,ID_QUERY,ID_SUPPORT]))

    print("*Macro F1 Average*: " + str(metrics.f1_score(resultsArr, predicted,average='macro',labels=[ID_COMMENT,ID_DENY,ID_QUERY,ID_SUPPORT])))
    print("Micro F1 Average: " + str(metrics.f1_score(resultsArr, predicted,average='micro',labels=[ID_COMMENT,ID_DENY,ID_QUERY,ID_SUPPORT])))
    print("Weighted F1 Average: " + str(metrics.f1_score(resultsArr, predicted,average='weighted',labels=[ID_COMMENT,ID_DENY,ID_QUERY,ID_SUPPORT])))

    print("Confusion Matix") 
    print(metrics.confusion_matrix(resultsArr, predicted))

show_details = False
    
if show_details == True:
    testWithDevData('Random Forest',randomForestClassifier)
    testWithDevData('Logistic Regression',logisticRegressionClassifier)
    testWithDevData('SVM',linearSVCClassifier)
    testWithDevData('Neural Network',mlpClassifier)
    testWithDevData('Gradient Boost',gradientBoostClassifier)
    testWithDevData('Decision Tree Bagging',baggingClassifier)
    testWithDevData('AdaBoost',adaBoostClassifier)
testWithDevData('Voting',votingClassifier)

Voting Accuracy: 0.580096503337
---------------------------------------------------------------------------Without Comments
             precision    recall  f1-score   support

       deny       0.00      0.00      0.00        11
      query       0.00      0.00      0.00        28
    support       0.81      0.32      0.46        69

avg / total       0.52      0.20      0.29       108

*Macro F1 Average*: 0.152777777778
Micro F1 Average: 0.325925925926
Weighted F1 Average: 0.292824074074
---------------------------------------------------------------------------Includng Comments
             precision    recall  f1-score   support

    comment       0.67      0.98      0.79       173
       deny       0.00      0.00      0.00        11
      query       0.00      0.00      0.00        28
    support       0.81      0.32      0.46        69

avg / total       0.61      0.68      0.60       281

*Macro F1 Average*: 0.312475604996
Micro F1 Average: 0.679715302491
Weighted F1 Average: 0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [175]:
# #Write dev results to file & apply scorer

# ResultDict = {}
# i = 0
# while i < len(DevTweetsIDs):
#     ResultDict[DevTweetsIDs[i]] = target_names[predicted[i]]
#     i+= 1
    
# with open(SubTaskADevResultsFilePath, 'w') as outfile:
#     json.dump(ResultDict, outfile)

# ipy = get_ipython()
# ipy.magic("run ScorerA.py \"" + SubTaskADevFilePath + "\" \"" + SubTaskADevResultsFilePath + "\"")

In [176]:
# print(type(devFts))
TrainAndDevCategories = TrainTweetsCategories+DevTweetsCategories
TrainAndDevFts = sp.vstack((trainFts,devFts),format='csr')
# print(trainFts.shape)
# print(devFts.shape)
# print(TrainAndDevFts.shape)

CategoriesArr = np.array(list(TrainAndDevCategories), dtype=int)

clf.fit(TrainAndDevFts,CategoriesArr)


VotingClassifier(estimators=[('rfClf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min...e,
            splitter='best'),
          learning_rate=1.0, n_estimators=100, random_state=None))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [177]:
# #Write test results to file

print(testFts.shape)
print(len(TestTweetsIDs))

predicted = clf.predict(testFts)

ResultDict = {}
i = 0
while i < len(TestTweetsIDs):
    ResultDict[TestTweetsIDs[i]] = target_names[predicted[i]]
    TestData[i][ATTR_CATEGORY] = predicted[i]
    i+= 1
    
with open(SubTaskATestResultsFilePath, 'w') as outfile:
    json.dump(ResultDict, outfile)

(1049, 10)
1049


In [178]:
#Write gold results to file & apply scorer

%run getScoreA.py

def getGoldResultsScoreA(name,classifier):

    classifier.fit(TrainAndDevFts,CategoriesArr)
    
    predicted = classifier.predict(goldFts)

    ResultDict = {}
    i = 0
    while i < len(GoldTweetsIDs):
        ResultDict[GoldTweetsIDs[i]] = target_names[predicted[i]]
        i+= 1

    with open(SubTaskAGoldResultsFilePath, 'w') as outfile:
        json.dump(ResultDict, outfile)

    # ipy = get_ipython()
    # ipy.magic("run ScorerA.py \"" + SubTaskAGoldFilePath + "\" \"" + SubTaskAGoldResultsFilePath + "\"")


    print( name + ' score: ' + str(getScoreA(SubTaskAGoldFilePath,SubTaskAGoldResultsFilePath)))

show_details = True
    
if show_details == True:
    getGoldResultsScoreA('Random Forest',randomForestClassifier)
    getGoldResultsScoreA('Logistic Regression',logisticRegressionClassifier)
    getGoldResultsScoreA('SVM',linearSVCClassifier)
    getGoldResultsScoreA('Neural Network',mlpClassifier)
    getGoldResultsScoreA('Gradient Boost',gradientBoostClassifier)
    getGoldResultsScoreA('Decision Tree Bagging',baggingClassifier)
    getGoldResultsScoreA('AdaBoost',adaBoostClassifier)
getGoldResultsScoreA('Voting',votingClassifier)

# getGoldResultsScore(logisticRegressionClassifier)
# getGoldResultsScore(decisionTreeClassifier)
# getGoldResultsScore(mlpClassifier)
# getGoldResultsScore(gradientBoostClassifier)
# getGoldResultsScore(baggingClassifier)
# getGoldResultsScore(adaBoostClassifier)
# getGoldResultsScore(votingClassifier)



Random Forest score: 0.6854146806482364
Logistic Regression score: 0.7645376549094376
SVM score: 0.765490943755958
Neural Network score: 0.7778836987607245
Gradient Boost score: 0.782650142993327
Decision Tree Bagging score: 0.67302192564347
AdaBoost score: 0.6367969494756911
Voting score: 0.7740705433746425


In [179]:
#Fooling around

#predicted[18]
#predicted[50]
#predicted

#import numpy
#numpy.where( predicted == 2)

In [180]:
#Fooling around

#list(DevTweetsCategories)[18]
#list(DevTweetsCategories)[50]
#DevData.values()

#print (list(DevTweetsText)[3])
#numpy.where( resultsArr == 2)

In [181]:
#Prediction for 1 Test

#d = ["Reports of shooting at  Dammartin en Goele on route N2 north east of Paris - French media says car chase under way"]
#x = count_vect.transform(d)
#p = clf.predict(x)
#for doc, category in zip(d, p):
#    print('%r => %s' % (doc, category))

#xx = count_vect.transform(d).toarray()
#print (numpy.where( xx != 0))

#count_vect.vocabulary_
#for kk,vv in count_vect.vocabulary_.items():
    #if vv == 663:
    #    print (kk)

#230 considered questions and are queries
#469 considered questions but are not queries
#98 are queries but not considered questions
    
Data = TrainData
#Data = DevData

count = 0
for tweet in Data:
    if tweet[QUEST_WORD_FEATURE] == 1 and tweet["category"] == ID_QUERY:
        count = count + 1
# print(count)

# for tweet in Data:
#     if tweet["category"] == ID_QUERY:
#         print('[%r] %r (%s,%s) -> %s (+%s,-%s,%s)' % (tweet["id"],tweet["text"],tweet["category"],tweet[QUEST_WORD_FEATURE],tweet[SENTIMENT_WORD_FEATURE],tweet[POS_WORDS_COUNT],tweet[NEG_WORDS_COUNT],tweet[NEUT_WORDS_COUNT]))

In [182]:
#Fooling around: Testing Tokenizing
# tweetText = "@UlrichJvV @affinity292 how many 'nazis' did leebowitz kill ?https://t.co/bZdlBn9qiF"
# tokens = tknzr.tokenize(tweetText)
# print(tokens)
# if (Question_Words_Set.intersection(tokens)):
#     print('Question')
# else:
#     print('not Question')



In [183]:
#Support Results
# i = 0
# r = 0
# a = 0
# while i < len(DevTweetsText):
#     if DevTweetsCategories[i] == ID_SUPPORT:
#         a = a + 1
#         if predicted[i] == ID_SUPPORT:
#             r = r + 1
#     i = i + 1

# print('%s => %s' % (a,r))

# i = 0
# while i < len(DevTweetsText):
#     if DevTweetsCategories[i] == ID_SUPPORT:
#         print('[%r] %r (%s) => %s (S=%s)' % (DevTweetsIDs[i],DevTweetsText[i], DevTweetsCategories[i],predicted[i],DevData[i][SUPPORT_WORD_FEATURE] ))
#     i = i +1

In [184]:
#Deny Results

# i = 0
# r = 0
# a = 0
# while i < len(DevTweetsText):
#     if DevTweetsCategories[i] == ID_DENY:
#         a = a + 1
#         if predicted[i] == ID_DENY:
#             r = r + 1
#     i = i + 1

# print('%s => %s' % (a,r))

# i = 0
# while i < len(DevTweetsText):
#     if DevTweetsCategories[i] == ID_DENY:
#         print('[%r] %r (%s) => %s (D=%s)' % (DevTweetsIDs[i],DevTweetsText[i], DevTweetsCategories[i],predicted[i],DevData[i][DENY_WORD_FEATURE] ))
#     i = i +1


'''
i = 0
while i < len(TrainTweetsText):
    if TrainTweetsCategories[i] == ID_DENY:
        print('[%r] %r (%s) => (D=%s)' % (TrainTweetsIDs[i],TrainTweetsText[i], TrainTweetsCategories[i],TrainData[i][DENY_WORD_FEATURE] ))
    i = i +1
'''

"\ni = 0\nwhile i < len(TrainTweetsText):\n    if TrainTweetsCategories[i] == ID_DENY:\n        print('[%r] %r (%s) => (D=%s)' % (TrainTweetsIDs[i],TrainTweetsText[i], TrainTweetsCategories[i],TrainData[i][DENY_WORD_FEATURE] ))\n    i = i +1\n"

In [185]:
#Query Results
# print("Query Results")
# i = 0
# r = 0
# a = 0
# while i < len(DevTweetsText):
#     if DevTweetsCategories[i] == ID_QUERY:
#         a = a + 1
#         if predicted[i] == ID_QUERY:
#             r = r + 1
#     i = i + 1

# print('%s => %s' % (a,r))

# i = 0
# while i < len(DevTweetsText):
#     if DevTweetsCategories[i] == ID_QUERY:
#         print('[%r] %r (%s) => %s (Q=%s)' % (DevTweetsIDs[i],DevTweetsText[i], DevTweetsCategories[i],predicted[i],DevData[i][QUEST_WORD_FEATURE] ))
#     i = i +1